## Filter Function 
`filter()` function is used to filter the rows from RDD/DataFrame based on the given condition or SQL expression, you can also use `where()` clause instead of the `filter()` if you are coming from an SQL background, both these functions operate exactly the same

Create Test Dataframe

In [21]:
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("accumulator").getOrCreate()

data = [
    (("James","","Smith"),["Java","Scala","C++"],"IL","M"),
    (("Anna","Rose",""),["Spark","Java","C++"],"NY","F"),
    (("Julia","","Williams"),["CSharp","VB"],"IL","F"),
    (("Maria","Anne","Jones"),["CSharp","VB"],"NY","M"),
    (("Jen","Mary","Brown"),["CSharp","VB"],"NY","M"),
    (("Mike","Mary","Williams"),["Python","VB"],"IL","M")
 ]
        
schema = StructType([
            StructField('name', StructType([
                StructField('firstname', StringType(), True),
                StructField('middlename', StringType(), True),
                StructField('lastname', StringType(), True)])),
            StructField('languages', ArrayType(StringType()), True),
            StructField('state', StringType(), True),
            StructField('gender', StringType(), True)])

df = spark.createDataFrame(data = data, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|IL   |M     |
|{Anna, Rose, }        |[Spark, Java, C++]|NY   |F     |
|{Julia, , Williams}   |[CSharp, VB]      |IL   |F     |
|{Maria, Anne, Jones}  |[CSharp, VB]      |NY   |M     |
|{Jen, Mary, Brown}    |[CSharp, VB]      |NY   |M     |
|{Mike, Mary, Williams}|[Python, VB]      |IL   |M     |
+----------------------+------------------+-----+------+



### DataFrame filter() with Column Condition
Use Column with the condition to filter the rows from DataFrame, using this you can express complex condition by referring column names using `df.colname`

In [22]:
# Using equals condition
df.filter(df.state == "IL").show(truncate=False)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|IL   |M     |
|{Julia, , Williams}   |[CSharp, VB]      |IL   |F     |
|{Mike, Mary, Williams}|[Python, VB]      |IL   |M     |
+----------------------+------------------+-----+------+



In [23]:
# not equals condition
df.filter(df.state != "IL").show(truncate=False) 
df.filter(~(df.state == "IL")).show(truncate=False)

+--------------------+------------------+-----+------+
|name                |languages         |state|gender|
+--------------------+------------------+-----+------+
|{Anna, Rose, }      |[Spark, Java, C++]|NY   |F     |
|{Maria, Anne, Jones}|[CSharp, VB]      |NY   |M     |
|{Jen, Mary, Brown}  |[CSharp, VB]      |NY   |M     |
+--------------------+------------------+-----+------+

+--------------------+------------------+-----+------+
|name                |languages         |state|gender|
+--------------------+------------------+-----+------+
|{Anna, Rose, }      |[Spark, Java, C++]|NY   |F     |
|{Maria, Anne, Jones}|[CSharp, VB]      |NY   |M     |
|{Jen, Mary, Brown}  |[CSharp, VB]      |NY   |M     |
+--------------------+------------------+-----+------+



Same example can also written as below. In order to use this first you need to import from `pyspark.sql.functions import col`

In [24]:
#Using SQL col() function
from pyspark.sql.functions import col

df.filter(col("state") == "IL").show(truncate=False) 

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|IL   |M     |
|{Julia, , Williams}   |[CSharp, VB]      |IL   |F     |
|{Mike, Mary, Williams}|[Python, VB]      |IL   |M     |
+----------------------+------------------+-----+------+



### DataFrame filter() with SQL Expression
filter DataFrame rows with SQL expressions

In [25]:
#Using SQL Expression
df.filter("gender == 'M'").show()
#For not equal
df.filter("gender != 'M'").show()
df.filter("gender <> 'M'").show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   IL|     M|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
|{Mike, Mary, Will...|      [Python, VB]|   IL|     M|
+--------------------+------------------+-----+------+

+-------------------+------------------+-----+------+
|               name|         languages|state|gender|
+-------------------+------------------+-----+------+
|     {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
|{Julia, , Williams}|      [CSharp, VB]|   IL|     F|
+-------------------+------------------+-----+------+

+-------------------+------------------+-----+------+
|               name|         languages|state|gender|
+-------------------+------------------+-----+------+
|     {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
|{Julia, , William

### Filter with Multiple Conditions
to `filter()` rows on DataFrame based on multiple conditions, you case use either `Column` with a condition or SQL expression. Below is just a simple example using `AND (&)` condition, you can extend this with `OR(|)`, and `NOT(!)` conditional expressions as needed

In [26]:
df.filter( (df.state  == "IL") & (df.gender  == "M")).show(truncate=False)  

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|IL   |M     |
|{Mike, Mary, Williams}|[Python, VB]      |IL   |M     |
+----------------------+------------------+-----+------+



### Filter Based on List Values
If you have a list of elements and you wanted to filter that is not in the list or in the list, use `isin()` function of Column class and it doesn’t have `isnotin()` function but you do the same using not operator `(~)`

In [27]:
li=["IL","CA","DE"]
df.filter(df.state.isin(li)).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   IL|     M|
| {Julia, , Williams}|      [CSharp, VB]|   IL|     F|
|{Mike, Mary, Will...|      [Python, VB]|   IL|     M|
+--------------------+------------------+-----+------+



Filter NOT IS IN List values, these show all records with NY (NY is not part of the list)

In [28]:
df.filter(~df.state.isin(li)).show()


+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
+--------------------+------------------+-----+------+



In [29]:
df.filter(df.state.isin(li)==False).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
+--------------------+------------------+-----+------+



### Filter Based on Starts With, Ends With, Contains
filter DataFrame rows by using `startswith()`, `endswith()` and `contains()` methods of Column class

In [30]:
# Using startswith
df.filter(df.state.startswith("N")).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|      {Anna, Rose, }|[Spark, Java, C++]|   NY|     F|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
+--------------------+------------------+-----+------+



In [31]:
#using endswith
df.filter(df.state.endswith("L")).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   IL|     M|
| {Julia, , Williams}|      [CSharp, VB]|   IL|     F|
|{Mike, Mary, Will...|      [Python, VB]|   IL|     M|
+--------------------+------------------+-----+------+



In [32]:
#contains
df.filter(df.state.contains("L")).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   IL|     M|
| {Julia, , Williams}|      [CSharp, VB]|   IL|     F|
|{Mike, Mary, Will...|      [Python, VB]|   IL|     M|
+--------------------+------------------+-----+------+



### Filter like and rlike
If you have SQL background you must be familiar with `like` and `rlike` (regex like), PySpark also provides similar methods in Column class to filter similar values using wildcard characters. You can use rlike() to filter by checking values case insensitive.

In [33]:
data2 = [(2,"Michael Smith"),(3,"Robert Williams"),
         (4,"John Smith"),(5,"George Smith")]

df2 = spark.createDataFrame(data = data2, schema = ["id","name"])

# like - SQL LIKE pattern
df2.filter(df2.name.like("%Smith%")).show()

+---+-------------+
| id|         name|
+---+-------------+
|  2|Michael Smith|
|  4|   John Smith|
|  5| George Smith|
+---+-------------+



In [34]:
# rlike - SQL RLIKE pattern (LIKE with Regex)
# This check case insensitive

df2.filter(df2.name.rlike("(?i)^*smith$")).show()

+---+-------------+
| id|         name|
+---+-------------+
|  2|Michael Smith|
|  4|   John Smith|
|  5| George Smith|
+---+-------------+



### Filter on an Array column
When you want to filter rows from DataFrame based on value present in an array collection column, you can use the first syntax. The below example uses `array_contains()` from Pyspark SQL functions which checks if a value contains in an array if present it returns true otherwise false.

In [35]:
from pyspark.sql.functions import array_contains

df.filter(array_contains(df.languages,"Java")).show(truncate=False) 

+----------------+------------------+-----+------+
|name            |languages         |state|gender|
+----------------+------------------+-----+------+
|{James, , Smith}|[Java, Scala, C++]|IL   |M     |
|{Anna, Rose, }  |[Spark, Java, C++]|NY   |F     |
+----------------+------------------+-----+------+



### Filtering on Nested Struct columns
If your DataFrame consists of nested struct columns, you can use any of the below syntaxes to filter the rows based on the nested column.

In [36]:
df.filter(df.name.lastname == "Williams").show(truncate=False)

+----------------------+------------+-----+------+
|name                  |languages   |state|gender|
+----------------------+------------+-----+------+
|{Julia, , Williams}   |[CSharp, VB]|IL   |F     |
|{Mike, Mary, Williams}|[Python, VB]|IL   |M     |
+----------------------+------------+-----+------+

